In [1]:
import torch
from torchvision import models
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image

In [2]:
resnet = models.resnet101(pretrained=True)

/Users/parkmiryeong/anaconda3/envs/midoi/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/parkmiryeong/anaconda3/envs/midoi/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
# 훈련에 사용하는 이미지 형식과 일치하게 만들기 위한 이미지 변환 작업
preprocess = transforms.Compose([
    transforms.Resize(256), # 입력 이미지의 크기를 256x256으로 조정
    transforms.CenterCrop(224), # 중심으로부터 224x224로 잘라내기
    transforms.ToTensor(), # 다차원 배열인 텐서 형태로 전환
    transforms.Normalize( # 지정된 평균과 표준편차를 가지도록 RGB를 정규화
        mean = [0.485, 0.485, 0.485],
        std = [0.229, 0.229, 0.229]
    )
])
preprocess

Compose(
    Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.485, 0.485, 0.485], std=[0.229, 0.229, 0.229])
)

In [5]:
# Inference
# 추론을 수행하기 위해서는 신경망을 eval 모드로 설정해야한다.
resnet.eval()
# eval 모드 설정을 빠뜨리면, 배치정규화나 드랍아웃과 같은 몇몇 사전학습된 모델은 이상한 결과를 만들어낸다.

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
img = Image.open('/Users/parkmiryeong/Desktop/연구인턴/무제 폴더/machine-learning/파이토치딥러닝마스터/1583915988281.jpg')
img_t = preprocess(img) # 전처리 파이프라인 통과
batch_t = torch.unsqueeze(img_t, 0) # 텐서를 reshape 하고 잘라낸 후 정규화해서 신경망에 넣을 수 있도록 준비
print(img_t.shape, '에서', batch_t.shape, '로 변경')

torch.Size([3, 224, 224]) 에서 torch.Size([1, 3, 224, 224]) 로 변경


In [7]:
out = resnet(batch_t)
out # 100개 클래스의 점수

tensor([[-2.8451e+00,  1.1356e+00, -4.2012e+00, -3.6197e+00, -3.2568e+00,
         -3.4103e-01, -4.4753e+00,  1.0596e+00,  2.4728e+00, -1.5030e+00,
          1.6606e+00, -4.1213e-01,  9.4110e-01,  5.7797e-01, -1.6172e+00,
          1.3185e+00, -5.5441e-01, -1.2074e+00, -1.3635e+00, -1.5370e-01,
         -2.8880e+00, -2.5669e+00, -3.1714e+00, -1.7627e+00, -1.6007e+00,
         -2.2592e+00,  6.4649e-01,  5.7910e-01, -1.0122e+00,  2.3671e+00,
         -2.0045e+00,  6.3623e-01,  8.6418e-01, -1.9047e+00, -8.9172e-01,
         -9.5513e-01,  1.2152e+00, -7.5385e-02,  2.4019e+00, -8.0000e-02,
          1.2331e+00,  1.2104e+00,  1.3750e+00,  2.0103e-01,  8.4921e-01,
         -4.4254e-01,  6.2430e-02,  1.0996e+00, -2.4287e+00, -1.8378e+00,
         -2.4888e+00,  1.6369e+00, -2.6193e-01, -3.4219e+00, -7.3185e-01,
         -3.5001e+00,  2.0857e-01, -1.5939e+00, -3.6055e+00, -1.2807e+00,
          5.5058e-01,  1.2294e+00,  2.4926e-02, -5.2477e-01, -1.1428e+00,
         -2.6390e+00,  3.0994e+00,  1.

In [8]:
maxpoint, index = torch.max(out, 1) # 텐서에서 최댓값과 이 최댓값이 들어있는 부분의 인덱스 출력
print(maxpoint, index) # 인덱스는 단순한 파이썬 숫자가 아닌, 1개 요소를 가지는 1차원 텐서이다.

tensor([15.4856], grad_fn=<MaxBackward0>) tensor([330])


In [9]:
# 출력을 [0, 1] 사이 값으로 정규화 하고 전체 합으로 나누기 위해 softmax 사용
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
percentage[index[0]].item() # item(): 하나의 원소를 가진 텐서에서 해당 값을 python 숫자로 추출 (여러 원소일경우 오류)

94.61666107177734

In [10]:
# 두번째, 세번째 점수도 알고싶다
_, indices = torch.sort(out, descending=True)
print([percentage[idx].item() for idx in indices[0][:5]])

[94.61666107177734, 4.969268798828125, 0.27201223373413086, 0.015071383677423, 0.010563577525317669]


### squeeze 와 unsqueeze

In [20]:
# squeeze : 텐서에서 크기가 1인 차원을 제거하여 차원을 축소한다. (인덱스 지정하면 해당 차원이 제거됨)
tensor = torch.randn(1, 3, 1)
squeezed_t = tensor.squeeze(dim=2) # dim: 특정 차원 인덱스를 지정

print('original tensor:', tensor.size())
print('squeezed tensor:', squeezed_t.size())

original tensor: torch.Size([1, 3, 1])
squeezed tensor: torch.Size([1, 3])


In [17]:
# unsqueeze : 특정 위치에 크기가 1인 새로운 차원 추가, 특정 차원의 인덱스 지정
tensor = torch.randn(3)
unsqueezed_t = tensor.unsqueeze(0)

print('original tensor:', tensor.size())
print('unsqueezed tensor:', unsqueezed_t.size())

original tensor: torch.Size([3])
unsqueezed tensor: torch.Size([1, 3])
